# ~ TilStat scraper ~

Utility to scrape the scores to enter the various courses over the years. Like any scrape utility, this code sucks.

In [8]:
import requests
from bs4 import BeautifulSoup as bs
from multiprocessing import Pool
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm.notebook import trange, tqdm

## Graduatorie V2 (Ingegneria dal 2022)

<b>Info importante:</b> la convenzione Politecnica è di riferirsi all'anno accademico con <b>l'ultimo anno del biennio</b>. Quindi il 2021/22 è definito come 2022.

Da quest'anno il formato delle statistiche è stato aggiornato. I livelli ora sono:

1.) Elenco generale, es: http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_a_acc=2022 <br>
1.1) Elenco per corso, es: http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_a_acc=2022&p_mnemo=ECE1T3 <br>
1.1.1) Singola graduatoria, es: http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_data=07/09/2021&p_a_acc=2022&p_mnemo=ECE1T3
 

In [9]:
base = 'http://didattica.polito.it'

In [10]:
indexes = []

# legge gli elenchi generali (1.) di normali e contingentati
for year in [2025]:
    indexes += [
        # normali
        {'uri':f'{base}/pls/portal30/preimma.pkg_grad.ing_sep?p_a_acc={year}',
         'conting': False, 'course': 'Ingegneria', 'year':year},
        
        # contingentati
        # {'uri':f'{base}/portal30/preimma.pkg_grad.ing_sep_cont?p_a_acc={year}',
        #  'conting': True, 'course': 'Ingegneria', 'year':year},
    ]
    
# partendo dagli elenchi generali, legge gli elenchi per corso
explode = []
for index in indexes:
    print(index['uri'])
    html = requests.get(index['uri']).text
    soup = bs(html, 'html.parser')
    
    subindex = [link for link in soup.find_all("a")
                     if "href" in link.attrs and 'p_mnemo=' in link['href'] and "L-" in link.text]
    
    for s in subindex:
        explode.append({
            **index,
            'corso unico': s.text, # .text.split('(')[0].strip(),
            'uri': base+s['href']
        })
        
# per analogia con gli anni precedenti, rinomino la variabile
indexes = explode
assert len(indexes) > 0

http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_a_acc=2025


In [11]:
# memorizzo i link di ciascuna graduatoria (rank)

ranks = []
for index in tqdm(indexes):
    html = requests.get(index['uri']).text
    soup = bs(html, 'html.parser')
    
    ranklinks = [link for link in soup.find_all("a")
                     if "href" in link.attrs and 'preimma' in link['href'] and link.text.startswith('Graduatoria del ')]
    for i, link in enumerate(ranklinks):
        
        # paranoid check
        # Es: Ingegneria has .ing string in each rank
        assert f".{index['course'].lower()[:3]}" in link['href'], "Wrong link assoc..."
        
        date = [int(x) for x in link.text.replace("Graduatoria del ","").split('/')]
        date = datetime(date[2],date[1],date[0])
        
        # paranoid check
        first_sane_rank = datetime(int(index['year'])-1, 7, 1) # dal primo luglio del primo anno di inizio coorte
        last_sane_rank = datetime(int(index['year']), 3, 15) # a metà marzo del secondo anno di inizio coorte
        assert first_sane_rank < date < last_sane_rank, "Wrong year assoc..."
        
        ranks.append({
            'uri': base+link['href'],
            'conting': index['conting'],
            'course': index['course'],
            'corso unico': index['corso unico'],
            'year': index['year'],
            'grad_n': i,
            'date': date
        })

  0%|          | 0/23 [00:00<?, ?it/s]

In [12]:
# printa alcune cose, per controllare che sia tutto ok prima del task lungo
ranks[10]

{'uri': 'http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_data=15/10/2024&p_a_acc=2025&p_mnemo=CEE1T1',
 'conting': False,
 'course': 'Ingegneria',
 'corso unico': 'CIVIL AND ENVIRONMENTAL ENGINEERING (L-7)',
 'year': 2025,
 'grad_n': 10,
 'date': datetime.datetime(2024, 10, 15, 0, 0)}

In [13]:
ranks

[{'uri': 'http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_data=03/09/2024&p_a_acc=2025&p_mnemo=CEE1T1',
  'conting': False,
  'course': 'Ingegneria',
  'corso unico': 'CIVIL AND ENVIRONMENTAL ENGINEERING (L-7)',
  'year': 2025,
  'grad_n': 0,
  'date': datetime.datetime(2024, 9, 3, 0, 0)},
 {'uri': 'http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_data=10/09/2024&p_a_acc=2025&p_mnemo=CEE1T1',
  'conting': False,
  'course': 'Ingegneria',
  'corso unico': 'CIVIL AND ENVIRONMENTAL ENGINEERING (L-7)',
  'year': 2025,
  'grad_n': 1,
  'date': datetime.datetime(2024, 9, 10, 0, 0)},
 {'uri': 'http://didattica.polito.it/pls/portal30/preimma.pkg_grad.ing_sep?p_data=13/09/2024&p_a_acc=2025&p_mnemo=CEE1T1',
  'conting': False,
  'course': 'Ingegneria',
  'corso unico': 'CIVIL AND ENVIRONMENTAL ENGINEERING (L-7)',
  'year': 2025,
  'grad_n': 2,
  'date': datetime.datetime(2024, 9, 13, 0, 0)},
 {'uri': 'http://didattica.polito.it/pls/portal30/preimma.pkg_grad.i

In [14]:
# dataframe che conterrà ogni riga di ogni graduatoria di ogni anno
dfs = []

# giro sulle graduatorie ed inserisco ogni riga in dfs
# si potrebbe parallellizzare ma tende a bloccarsi, con una discreta connessione ci vanno 5 minuti
for rank in tqdm(ranks):
    
    html = requests.get(rank['uri']).text
    soup = bs(html, 'html.parser')
    
    headers = soup.select("table th")
    rows = soup.select("table tr")
    cells = soup.select("table td")

    # paranoid check
    assert len(headers)*(len(rows)-1) == len(cells), "Wrong table format"

    headervalues = [str(h.text.lower().strip()) for h in headers]
    values = [str(c.text.strip()) for c in cells]
    shapedvalues = np.asarray(values).reshape((len(rows)-1),len(headers)).astype(str)
    df = pd.DataFrame(shapedvalues, columns=headervalues)

    # remove special students
    df = df[df["totale"] != "Certificazione Internazionale"]
    
    df["totale"] = df["totale"].str.replace(',','.').astype(float)
    df['uri'] = rank['uri']
    df['conting'] = rank['conting']
    df['course'] = rank['course']
    df['corso unico'] = rank['corso unico']
    df['grad_n'] = rank['grad_n']
    df['date'] = rank['date']
    df['year'] = rank['year']
    
    dfs.append(df)

  0%|          | 0/299 [00:00<?, ?it/s]

In [15]:
result = pd.concat(dfs, ignore_index=True)

In [17]:
name = f"v2b_consolidato_{str(datetime.now()).replace(':','-')}.csv"
result.to_csv(name, sep=";")
name

'v2b_consolidato_2025-04-23 05-03-36.058723.csv'